In [6]:
# simply copying the files

from PIL import Image
import os
import shutil

# assign directory
path1 = "C:\\Users\\jacob\\aps360\\465_images"
path2 = "C:\\Users\\jacob\\aps360\\665_images"
path3 = "C:\\Users\\jacob\\aps360\\Grietas_darknet"

highest_list = ["C:\\Users\\jacob\\aps360\\dataset\\old_datasets\\465_images\\", "C:\\Users\\jacob\\aps360\\dataset\\old_datasets\\665_images\\", "C:\\Users\\jacob\\aps360\\dataset\\old_datasets\\Grietas_darknet\\"]
upper_list = ["train\\", "test\\", "valid\\"]
lower_list12 = ["images\\", "labels\\"]

final_path = "C:\\Users\\jacob\\aps360\\dataset\\final_dataset\\"
final_upper_list = ["data\\", "train\\", "test\\", "valid\\"]
final_lower_list = ["images\\", "labels\\"]

# iterate over files in that directory
image_counter = 0
label_counter = 0

def format_4(num):
    if (num >= 1000):
        return str(num)
    elif (num >= 100):
        return "0" + str(num)
    elif (num >= 10):
        return "00" + str(num)
    else:
        return "000" + str(num)

# iterate through each of the datasets
for path in highest_list:
    
    # iterate through each of the training/valid/test sets
    for dataset in upper_list:
        
        # if we are from the first 2 datasets, the format is different
        if (path in highest_list[:2]):
            
            for filename in os.listdir(path+dataset+lower_list12[0]):
                
                f = os.path.join(path+dataset+lower_list12[0], filename)
                # checking if it is a file
                image = Image.open(f)
                new_image = image.resize((400, 300))
                new_image.save(final_path+final_upper_list[0]+final_lower_list[0]+"yolo_{}.jpg".format(format_4(image_counter)))
                
                image_counter += 1
                
            for filename in os.listdir(path+dataset+lower_list12[1]):
                f = os.path.join(path+dataset+lower_list12[1], filename)
                shutil.copy(f, final_path+final_upper_list[0]+final_lower_list[1]+"yolo_{}.txt".format(format_4(label_counter)))
             
                label_counter += 1
                
        # the last dataset has different format
        else:
            
            for filename in os.listdir(path+dataset):
                
                if (filename[-3:] == "jpg"):
                    f = os.path.join(path+dataset, filename)
                    # checking if it is a file
                    image = Image.open(f)
                    new_image = image.resize((400, 300))
                    new_image.save(final_path+final_upper_list[0]+final_lower_list[0]+"yolo_{}.jpg".format(format_4(image_counter)))
                    
                    
                    image_counter += 1
                    
                elif (filename[-3:] == "txt"):
                    f = os.path.join(path+dataset, filename)
                    shutil.copy(f, final_path+final_upper_list[0]+final_lower_list[1]+"yolo_{}.txt".format(format_4(label_counter)))
                    
                    label_counter += 1
        
# ensure that all labels are 0 and not 1

for txt in os.listdir(final_path+final_upper_list[0]+final_lower_list[1]):
    with open(final_path+final_upper_list[0]+final_lower_list[1]+txt, "r") as src:
        with open(final_path+"temp.txt", "w") as dest:
            for line in src.readlines():
                line = line.split(" ")
                if (line[0] != "0"):
                    line[0] = "0"
                line = " ".join(line)
                dest.write(line)
    # write corrected file to collection
    shutil.copy(final_path+"temp.txt", final_path+final_upper_list[0]+final_lower_list[1]+txt)

In [8]:
from PIL import Image, ImageFilter, ImageEnhance
import os
import shutil
import random

path = "C:\\Users\\jacob\\aps360\\dataset\\final_dataset\\"
final_upper_list = ["data\\", "train\\", "test\\", "valid\\"]
final_lower_list = ["images\\", "labels\\"]

counter = 0
for counter in range(2492):
    f = os.path.join(path+final_upper_list[0]+final_lower_list[0], "yolo_{}.jpg".format(format_4(counter)))
    
    for i in range(5):
        # convert colour scheme
        image = Image.open(f)
        image = image.convert("RGB")
        d = image.getdata()

        new_image = []
        for item in d:
             new_image.append((int(item[0] * 0.85), int(item[1] * 1.05), int(item[2] * 1.1)))
        image.putdata(new_image)

        # image contrast
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(0.5+random.uniform(0,1.0))

        # image sharpness
        enhancer = ImageEnhance.Sharpness(image)
        image = enhancer.enhance(0.5+random.uniform(0,1.0))

        # image colour
        enhancer = ImageEnhance.Color(image)
        image = enhancer.enhance(0.5+random.uniform(0,1.0))

        # image brightness
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(0.5+random.uniform(0,1.0))

        # only flip half of the images
        if (i % 2 == 0):
            # flip image
            image = image.transpose(Image.FLIP_LEFT_RIGHT)
            # flip the .txt file
            with open(path+final_upper_list[0]+final_lower_list[1]+"yolo_{}.txt".format(format_4(counter)), "r") as src:
                with open(path+"temp.txt", "w") as dest:
                    for line in src.readlines():
                        line = line.split(" ")
                        line[1] = str(1-float(line[1]))
                        line = " ".join(line)
                        dest.write(line)  
        # other half aren't flipped
        else:
            with open(path+final_upper_list[0]+final_lower_list[1]+"yolo_{}.txt".format(format_4(counter)), "r") as src:
                with open(path+"temp.txt", "w") as dest:
                    for line in src.readlines():
                        dest.write(line)  

        image.save(final_path+final_upper_list[0]+final_lower_list[0]+"yolo_{}_aug{}.jpg".format(format_4(counter), i+1))
        shutil.copy(path+"temp.txt", final_path+final_upper_list[0]+final_lower_list[1]+"yolo_{}_aug{}.txt".format(format_4(counter), i+1))


In [9]:
path = "C:\\Users\\jacob\\aps360\\dataset\\final_dataset\\"
final_upper_list = ["data\\", "train\\", "test\\", "valid\\"]
final_lower_list = ["images\\", "labels\\"]

counter = 0
for filename in os.listdir(path+final_upper_list[0]+final_lower_list[0]):
    f = os.path.join(path+final_upper_list[0]+final_lower_list[0], filename)
    if (counter % 60) in range(0,6):
        shutil.copy(f, path+final_lower_list[0]+final_upper_list[2]+filename)
    elif (counter % 60) in range(6,12):
        shutil.copy(f, path+final_lower_list[0]+final_upper_list[3]+filename)
    else:
        shutil.copy(f, path+final_lower_list[0]+final_upper_list[1]+filename)
    counter += 1
    
counter = 0
for filename in os.listdir(path+final_upper_list[0]+final_lower_list[1]):
    f = os.path.join(path+final_upper_list[0]+final_lower_list[1], filename)
    if (counter % 60) in range(0,6):
        shutil.copy(f, path+final_lower_list[1]+final_upper_list[2]+filename)
    elif (counter % 60) in range(6,12):
        shutil.copy(f, path+final_lower_list[1]+final_upper_list[3]+filename)
    else:
        shutil.copy(f, path+final_lower_list[1]+final_upper_list[1]+filename)
    counter += 1



In [10]:
import os

counter_dict = {}

path = "C:\\Users\\jacob\\aps360\\dataset\\final_dataset\\"
final_upper_list = ["train\\", "test\\", "valid\\"]
lower_list = "labels\\"

for dataset in final_upper_list:
    for filename in os.listdir(path+lower_list+dataset):
        with open(path+lower_list+dataset+filename, "r") as f:
            potholes = len(f.readlines())
            if potholes in counter_dict:
                counter_dict[potholes] += 1
            else:
                counter_dict[potholes] = 1

# get length
length = 0
for num in counter_dict:
    length += counter_dict[num]

# get mean
num_sum = 0
for num in counter_dict:
    num_sum += counter_dict[num] * num
mean = num_sum / length

# get std_dev
num_sum_squared = 0
for num in counter_dict:
    num_sum_squared += counter_dict[num] * (num - mean) ** 2
std_dev = (num_sum_squared / length) ** 0.5

In [11]:
print(mean, std_dev)
print(counter_dict)
print("\n")
for num in range(len(counter_dict)):
    print(num, counter_dict[num])

2.565008025682183 3.0192313434436486
{2: 2712, 3: 1578, 1: 4524, 7: 324, 4: 846, 5: 708, 6: 282, 13: 240, 9: 180, 8: 132, 17: 24, 10: 204, 15: 12, 14: 18, 19: 12, 12: 138, 11: 174, 0: 2826, 18: 12, 16: 6}


0 2826
1 4524
2 2712
3 1578
4 846
5 708
6 282
7 324
8 132
9 180
10 204
11 174
12 138
13 240
14 18
15 12
16 6
17 24
18 12
19 12


In [4]:
from PIL import Image
import os
import shutil

def format_2(num):
    if (num >= 10):
        return str(num)
    else:
        return "0" + str(num)

image_counter = 0
for filename in os.listdir("C:\\Users\\jacob\\aps360\\dataset\\manual\\images\\"):
                
    f = os.path.join("C:\\Users\\jacob\\aps360\\dataset\\manual\\images\\", filename)
    # checking if it is a file
    image = Image.open(f)
    new_image = image.resize((400, 300))
    new_image.save("C:\\Users\\jacob\\aps360\\dataset\\manual\\images\\"+"yolo_test_{}.jpg".format(format_2(image_counter)))

    image_counter += 1

In [8]:
label_counter = 0
for i in range(27):
    f = os.path.join("C:\\Users\\jacob\\aps360\\dataset\\final_dataset\\", "temp.txt")
    shutil.copy(f, "C:\\Users\\jacob\\aps360\\dataset\\manual\\labels\\" + "yolo_test_{}.txt".format(format_2(label_counter)))
    label_counter += 1

In [148]:
from PIL import Image, ImageDraw

def show_bbox(image_path):
    # convert image path to label path
    label_path = image_path.replace('/images/', '/labels/')
    label_path = label_path.replace('.jpg', '.txt')

    # Open the image and create ImageDraw object for drawing
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    with open(label_path, 'r') as f:
        for line in f.readlines():
            # Split the line into five values
            label, x, y, w, h = line.split(' ')

            # Convert string into float
            x = float(x)
            y = float(y)
            w = float(w)
            h = float(h)

            # Convert center position, width, height into
            # top-left and bottom-right coordinates
            W, H = image.size
            x1 = (x - w/2) * W
            y1 = (y - h/2) * H
            x2 = (x + w/2) * W
            y2 = (y + h/2) * H

            # Draw the bounding box with red lines
            draw.rectangle((x1, y1, x2, y2),
                           outline=(255, 0, 0), # Red in RGB
                           width=5)             # Line width
    image.show()
    

path = "C:/Users/jacob/aps360/dataset/manual/images/yolo_test_26.jpg"
show_bbox(path)